In [ ]:
# !git clone 'https://github.com/Lorenz92/SKADC1.git'
# % cd SKADC1
# !echo $PWD

In [ ]:
import pandas as pd
import numpy as np

import src.dataset as dataset
import src.config as config 
from src.utils import *
import src.models as models
import src.losses as loss

path = config.TRAIN_PATCHES_FOLDER
%load_ext autoreload
%autoreload 2

np.random.seed(config.RANDOM_SEED)

In [ ]:
if 'google.colab' in str(get_ipython()):
  use_colab = True
  print('Running on CoLab')
else:
  use_colab = False
  print('Not running on CoLab')

In [ ]:
if use_colab:
    # Read file from Colab Notebook
    from google.colab import drive
    drive.mount('/content/drive')
    config.MODEL_WEIGHTS = "/content/drive/My Drive/Colab Notebooks/SKADC1"
    config.IMAGE_PATH = "/content/drive/My Drive/Colab Notebooks/SKADC1/asset/560Mhz_1000h.fits"

In [ ]:
# Choose the feature extraction model
# backbone='baseline_44'
backbone='baseline_16'
# backbone='vgg16'

if backbone=='baseline_16':
    # config.patch_dim = 50
    config.patch_dim = 20
    config.resizePatch = True
    config.rpn_stride = 4
    config.num_rois = 16
    # anchors in the original image size
    config.anchor_box_scales = [4, 8, 16, 24, 32, 64]
    # config.anchor_box_scales = [6, 8, 12, 16, 24, 32, 64]
    config.resizeFinalDim = 100
    input_shape_1 = config.resizeFinalDim
elif backbone=='baseline_44':
    config.patch_dim = 20
    config.resizePatch = True
    config.rpn_stride = 8
    config.num_rois = 16
    # config.anchor_box_scales = [16,32,64] # anchors in the original image size
    config.anchor_box_scales = [4, 8, 16, 24, 32, 64]
    config.resizeFinalDim = 100
    input_shape_1 = config.resizeFinalDim
else:
    config.patch_dim = 100
    config.resizePatch = True
    config.rpn_stride = 16
    config.num_rois = 4
    config.resizeFinalDim = 600
    input_shape_1=config.resizeFinalDim
    config.anchor_box_scales = [32, 64, 128]


config.anchor_num = len(config.anchor_box_ratios)*len(config.anchor_box_scales)
input_shape_2=(None, 4)

print(config.resizePatch)
print(config.rpn_stride)

checkpoint = get_model_last_checkpoint(backbone)
print(f'Model last checkpoint: {checkpoint}')

file_path = f'{config.MODEL_WEIGHTS}/{backbone}'
print(f'Writing configuration on txt file: {config.MODEL_WEIGHTS}/config.txt')

if not os.path.exists(file_path):
        os.makedirs(file_path)
        
f = open(f'{file_path}/config.txt',"w+")
f.write(f'backbone = {backbone}\n config.patch_dim = {config.patch_dim}\n config.resizePatch = {config.resizePatch}\n config.rpn_stride = {config.rpn_stride}\n config.num_rois = {config.num_rois}\n config.anchor_box_scales = {config.anchor_box_scales}\n config.resizeFinalDim = {config.resizeFinalDim}\n input_shape_1 = {input_shape_1}')
f.close()

In [ ]:
# Dataset parsing and loading
# use "subset" in config file to load a small portion of data for development/debugging purposes
ska_dataset = dataset.SKADataset(k=3, print_info=False, show_plot=False, use_pb=False)

In [ ]:
ska_dataset.cleaned_train_df[['width', 'height', 'area_orig', 'area_cropped']].describe()

In [ ]:
ska_dataset.cleaned_train_df[['width', 'height', 'area_orig']].quantile([.1,.2,.3,.4,.5,.6,.7,.8,.9,.95,.98,.99,1.])

In [ ]:
objects_to_ignore=[20167150, 27514971]


In [ ]:
show_plot = True
ska_dataset.generate_patches(limit=10, plot_patches=show_plot, objects_to_ignore=objects_to_ignore, source_dir=None, rgb_norm=True)
# ska_dataset.generate_patches(limit=4000, plot_patches=show_plot, objects_to_ignore=objects_to_ignore, source_dir='./data/training/patches', rgb_norm=True)
# ska_dataset.generate_patches(limit=1000, plot_patches=show_plot, objects_to_ignore=objects_to_ignore, source_dir='./data/training/patches', rgb_norm=False)

In [ ]:
# ska_dataset.analyze_class_distribution()

In [ ]:
# ska_dataset.split_train_val_stratified(random_state=42, val_portion=0.2)

# print(len(ska_dataset.train_patch_list))
# print(len(ska_dataset.val_patch_list))


In [ ]:
ska_dataset.split_train_val(random_state=42, val_portion=0.2, balanced=False)

## datagen + calc_rpn -> rpn_net -> rpn_to_roi -> calc_iou -> cls_net

# Training

In [ ]:
# print_img(config.TRAIN_PATCHES_FOLDER, '3_16406_16729_20', show_data=True)


In [ ]:
# # Debugging

# train_patch_list = ska_dataset.train_patch_list
# patches_folder_path=config.TRAIN_PATCHES_FOLDER

# train_datagen = prep.get_anchor_gt(patches_folder_path, ['20_16396_16729_20'], backbone, pixel_mean=None)
# image, [y_rpn_cls_true, y_rpn_reg_true], img_data_aug, _, _, patch_id = next(train_datagen)



### Get FRCNN model

In [ ]:
rpn_model, detector_model, total_model = models.get_train_model(input_shape_1=input_shape_1, input_shape_2=input_shape_2, anchor_num=config.anchor_num, pooling_regions=config.pooling_regions, num_rois=config.num_rois, num_classes=len(ska_dataset.class_list)+1, backbone=backbone, use_expander=False)

rpn_model.summary()
detector_model.summary()
total_model.summary()

### Load weights

In [ ]:
ska_dataset.class_list

In [ ]:
# checkpoint='0_frcnn_baseline_36.h5'
models.load_weigths(rpn_model, detector_model, backbone, resume_train=False, checkpoint=checkpoint)
models.compile_models(rpn_model, detector_model, total_model, rpn_losses=[loss.rpn_loss_cls, loss.rpn_loss_regr], detector_losses=[loss.detector_loss_cls, loss.detector_loss_regr], class_list=ska_dataset.class_list)
# models.compile_models(rpn_model, detector_model, total_model, rpn_losses=[loss.rpn_loss_cls, loss.rpn_loss_regr], detector_losses=[loss.categorical_focal_loss([.25, .25, 0.05], 2.), loss.detector_loss_regr], class_list=ska_dataset.class_list)
# models.compile_models(rpn_model, detector_model, total_model, rpn_losses=[loss.rpn_loss_cls, loss.rpn_loss_regr], detector_losses=[loss.categorical_focal_loss([2, 0.5, 2, 2, 0.05], 2.), loss.detector_loss_regr], class_list=ska_dataset.class_list)



In [ ]:
# Specifically checking backbone weights

# total_model.weights[24:25][0][0][0][0]

In [ ]:
# Check that all of the pretrained weights have been loaded.
import numpy as np
for i, j in zip(total_model.weights, rpn_model.weights): 
    # print(i,j)
    assert np.allclose(i,j), 'Weights don\'t match!'

### Train

- errore "Exception: 'a' cannot be empty unless no samples are taken" quando nel sampling ci sono meno di 4 roi
- errore "None type object is not iterable" dovuto al max(IoUs) quando calc_iou torna None, None, None, None
- patch di debug 1550_16376_16779_100
- capire il parametro classifier_regr_std in che modo influenza il training

- provare normalizzazione valori immagini di input
- provare a far passare più roi anzichè 4

In [ ]:
len(ska_dataset.train_patch_list)

In [ ]:
# In this cell we modify the image for debugging purposes

# import cv2

print_img(config.TRAIN_PATCHES_FOLDER, '2_16396_16729_20')
### Flip image
# img_aug = np.load(f'{config.TRAIN_PATCHES_FOLDER}/2_16396_16729_20/2_16396_16729_20.npy')
# img_data_aug = pd.read_pickle(f'{config.TRAIN_PATCHES_FOLDER}/2_16396_16729_20/2_16396_16729_20.pkl')

# rows, cols = img_aug.shape[:2]

# img_aug = cv2.flip(img_aug, 0)
# for index, row in img_data_aug.iterrows():
#     y1 = row['y1s']
#     y2 = row['y2s']
#     img_data_aug.at[index,'y2s']= rows - y1
#     img_data_aug.at[index,'y1s']= rows - y2

# if not os.path.exists(f'{config.TRAIN_PATCHES_FOLDER}/2_16396_16729_20_aug/'):
#     os.makedirs(f'{config.TRAIN_PATCHES_FOLDER}/2_16396_16729_20_aug/')

# np.save((f'{config.TRAIN_PATCHES_FOLDER}/2_16396_16729_20_aug/2_16396_16729_20_aug.npy'), img_aug)
# img_data_aug.to_pickle(f'{config.TRAIN_PATCHES_FOLDER}/2_16396_16729_20_aug/2_16396_16729_20_aug.pkl')

# print_img(config.TRAIN_PATCHES_FOLDER, '2_16396_16729_20_aug')
###


In [ ]:
from src.train import *

val_patch_list = None
train_frcnn(rpn_model, detector_model, total_model, ['2_16396_16729_20'], val_patch_list, ska_dataset.class_list, num_epochs=100, patches_folder_path=config.TRAIN_PATCHES_FOLDER, backbone=backbone, resume_train=True)
# train_frcnn(rpn_model, detector_model, total_model, ska_dataset.train_patch_list, val_patch_list, ska_dataset.class_list, num_epochs=150, patches_folder_path=config.TRAIN_PATCHES_FOLDER, backbone=backbone, resume_train=True)

# Validation

In [ ]:
rpn_model_eval, detector_model_eval, total_model_eval = models.get_eval_model(input_shape_1=input_shape_1, input_shape_2=input_shape_2, input_shape_fmap=None, anchor_num=config.anchor_num, pooling_regions=config.pooling_regions, num_rois=config.num_rois, num_classes=len(ska_dataset.class_list)+1, backbone=backbone, use_expander=False)

rpn_model_eval.summary()
detector_model_eval.summary()
total_model_eval.summary()

In [ ]:
# Models used for mAP eval
cp = '66_frcnn_baseline_44.h5'
# cp = '0_frcnn_baseline_16.h5'
models.load_weigths(rpn_model_eval, detector_model_eval, backbone, checkpoint=cp)
models.compile_models(rpn_model_eval, detector_model_eval, total_model_eval, rpn_losses=[loss.rpn_loss_cls, loss.rpn_loss_regr], detector_losses=[loss.detector_loss_cls, loss.detector_loss_regr], class_list=ska_dataset.class_list)

In [ ]:
patch_id = ['2_16396_16729_20']
# patch_id = ska_dataset.train_patch_list
# print(patch_id)
# gt = pd.read_pickle(f'{config.TRAIN_PATCHES_FOLDER}/{patch_id[0]}/{patch_id[0]}.pkl')
# display(gt['class_label'])

preds, mAP, prec, recall = evaluate_model(rpn_model_eval, detector_model_eval, backbone, patch_id, ska_dataset.class_list, metric_threshold=.5, acceptance_treshold=.5)

In [ ]:
preds

In [ ]:
print(config.patch_dim , float(config.resizeFinalDim))

In [ ]:
print_img(config.TRAIN_PATCHES_FOLDER, '5_16501_16729_50')

In [ ]:
anch = pd.read_pickle(f'{config.EVAL_RESULTS}/2_16396_16729_20/2_16396_16729_20.pkl')
display(anch)

In [ ]:
anch['width'] = anch['x2s'] - anch['x1s']
anch['heght'] = anch['y2s'] - anch['y1s']

In [ ]:
anch.describe() 

In [ ]:
print_img(config.TRAIN_PATCHES_FOLDER, '2_16396_16729_20', config.EVAL_RESULTS)

TODO - 20210508:
- [X] troncamento rumore a 1e-6 tramite half gaussian

TODO - 20210605:
- [X] modificare RPNNet in modo che dia in output anche la backbone - Lorenzo
- [X] scrivere bene training loop: salvare le loss in un df su disco + salvare pesi modello ad ogni giro (Lorenzo - finire di debuggare)

TODO - 20210620:
- [x] implementare mAP in una funzione che prende come parametro un modello o i suoi pesi
- [x] implementare resNet50
- [x] implementare predicted rois - Lorenzo
- [X] implementare plot loss training - Lorenzo
- [X] finire classe datasetv2 - Alice
- [X] check se su colab le performance sono migliori - Lorenzo

TODO - 20210627
- [X] split dataset su combinazioni classi - Alice
- [x] provare campionamento random patch ed osservare le due distribuzioni - Alice

TODO - 20210703
- [x] sistemare salvataggio loss training loop - Lorenzo
- [x] Riscalare immagini tra 0-255 - Alice
- [x] capire se passare tre immagini diverse come input
- [x] usare media vgg16 per zero-centering - Alice

TODO - 20210705
- [x] sistemare nomi funzioni dataset per trasformazione rgb

TODO - 20210711
- [x] rifattorizzare classe dataset spostando nel costruttore i metodi che calcolano i suoi attributi - Lorenzo
- [x] chek valori pixel in input per resnet
- [x] fare funzione per plottare le predictions
- [ ] trainare tutto su colab

TODO - 20210714
- [x] ragionare su come scalare le immagini fra 0 e 1, attualmente hanno tanti valori schiacciati a 0 e il massimo su tutto il train a a 0.4

TODO - 20210717
- [ ] Ablation study: provare a rimuovere stage4 nella resnet - se c'è tempo
- [x] Provare con nostra pixel_mean e con vgg16 pixel_mean -> per il momento abbiamo scartato la prima opzione
- [x] Fare qualche analisi di distribuzione delle classi/dim box del dataset - Alice
- [x] Aggiungere normalizzazione dopo zero centering per resNet50, sulla base del range globale dell'immagine di training
- [x] Provare pulizia dataset originale sulla base del rumore/flusso - Alice
- [ ] Cambiare nomi di tutto - alla fine
- [x] implementare zero-centering su volare medio RGB delle nostre patch
- [x] Funzione che trova l'ultimo checkpoint in colab prima del load_weights - Lorenzo

TODO - 20210801
- [x] Debuggare training baseline 8 e 16 - L
- [x] Finire prove pulizia dataseet noise variando k - A

TODO - 20210904
- [ ] Provare NMS per rimuovere oggetti crowded
- [ ] In evaluate model eliminare le box con una delle due dim a 0 e quelle che escono dalla patch

1.  Summary
    - riassunto progetto
2.  Background
    - SoTA + teoria di base
3.  System Description
    - descrizione dei nostri modelli e dei loro componenti (moduli)
4.  Experimental setup and results
    - dataset pre processing
    - training environment
    - metrics
    - results
5.  Results and error analysis
6.  Discussion

# Plotting

In [ ]:
loss_history = np.load(f"./model/{backbone}/loss_history.npy")
print(loss_history.shape)

In [ ]:
loss_history

In [ ]:
plot_loss(loss_history[:])

In [ ]:
loss_history[:,1].shape

In [ ]:
np.convolve(loss_history[1], np.ones(10), 'valid') / 10

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

lsma_0 = moving_average(loss_history[100:,2], 200)


In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.arange(0, len(lsma_0)), lsma_0, 'r')
plt.title('rpn cls')

## Modelli da trainare:
epoch lenght = 200
- baseline_16: patch 50 -> 100, [8,16,32,64]-[1:1], 16roi, limit=4000, 10k epoche, focal_loss, norm: /255, no bilanciamento dataset, rgbNorm=True - A
- baseline_16: patch 50 -> 100, [8,16,32,64]-[1:1], 16roi, limit=4000, 10k epoche, NO focal_loss, norm: /255, no bilanciamento dataset, rgbNorm=True - A

- baseline_16: patch 20->100,  [8,16,32,64]-[1:1,1:2,2:1], 16roi, limit=4000, 10k epoche, focal_loss, norm: /255, no bilanciamento dataset, rgbNorm=True - L
- baseline_16: patch 20->100,  [8,16,32,64]-[1:1,1:2,2:1], 16roi, limit=4000, 10k epoche, focal_loss, norm: /255, no bilanciamento dataset, rgbNorm=True, layer 3 e 4 trainabili - L

- baseline_16: patch 100->100,  [8,16,32,64]-[1:1,1:2,2:1], 16roi, limit=4000, 10k epoche, focal_loss, norm: /255, no bilanciamento dataset, rgbNorm=True

- vgg16: 100 -> 600, [32,64,128,256,512]-[1:1,1:2,2:1], 16roi, limit=4000, 10k epoche, focal_loss, norm: /255, no bilanciamento dataset, rgbNorm=True